In [1]:
import random

import numpy as np


class Agent:
    def __init__(self, ini_room, connections, method):
        self.room = ini_room
        self.all_rooms = [room for room in connections.keys()]
        self.connections = connections
        self.history = []
        self.distribution = {room: 1 for room in self.all_rooms}
        self.next_distribution(self.room)
        self.method = method

    def __eq__(self, other):
        return self.room == other.room

    def auto_act(self):
        if self.method == 'random':
            self.random_act()
        elif self.method == 'distribution':
            self.act_with_distribution()

    def random_act(self):
        nr = random.choice(self.connections[self.room])
        self.move(nr)

    def act_with_distribution(self):
        distribution = self.normalize_distribution()
        nr = np.random.choice(self.connections[self.room], p=distribution)
        self.move(nr)

    def move(self, nr):
        self.history.append(self.room)
        self.room = nr
        self.next_distribution(self.room)

    def normalize_distribution(self):
        distribution = np.array([self.distribution[value] for value in self.connections[self.room]])
        # print(distribution)
        return distribution / np.sum(distribution)

    def next_distribution(self, room):
        new_distribution = {room: 0 for room in self.all_rooms}
        for pos_loc, next_locs in self.connections.items():
            if pos_loc != room:
                for next_loc in next_locs:
                    new_distribution[next_loc] += self.distribution[pos_loc]
        # Avoid extreme large value in distribution which might exceed the limitation of int
        if all(value > 1000 for value in list(new_distribution.values())):
            new_distribution = {key: value / 1000 for key, value in new_distribution.items()}
        self.distribution = new_distribution


class Problem:
    def __init__(self, start_room, connections):
        self.start_room = start_room
        self.connections = connections
        self.agent1 = None
        self.agent2 = None

    def init_problem(self, method1, method2):
        self.agent1 = Agent(self.start_room, self.connections, method1)
        self.agent2 = Agent(random.choice([room for room in self.connections.keys() if room != self.start_room]),
                            self.connections, method2)

    def auto_execute_a_round(self):
        self.agent1.auto_act()
        self.agent2.auto_act()

    def execute_a_round(self, room1, room2):
        self.agent1.move(room1)
        self.agent2.move(room2)

    def simulate(self, method1, method2, times=10000):
        total_moves = 0
        for _ in range(times):
            self.init_problem(method1, method2)
            while self.agent1.room != self.agent2.room:
                self.auto_execute_a_round()
                total_moves += 1
        print(
            f'method: {method1, method2}\nsimulate times: {times}\nTotal moves: {total_moves}\nAverage moves: {total_moves / times}\n---------')


method_pair = [('random', 'random'), ('distribution', 'random'), ('distribution', 'distribution')]


In [2]:
ini_room1 = 'A'
connections1 = {'A': ['A', 'B', 'D'],
                'B': ['A', 'B', 'C'],
                'C': ['B', 'C', 'D'],
                'D': ['A', 'C', 'D']}

problem1 = Problem(ini_room1, connections1)
for method1, method2 in method_pair:
    problem1.simulate(method1=method1, method2=method2)

method: ('random', 'random')
simulate times: 10000
Total moves: 45104
Average moves: 4.5104
---------
method: ('distribution', 'random')
simulate times: 10000
Total moves: 44241
Average moves: 4.4241
---------
method: ('distribution', 'distribution')
simulate times: 10000
Total moves: 44250
Average moves: 4.425
---------


In [3]:
ini_room2 = 'A'
connections2 = {'A': ['A', 'B', 'E', 'F'],
                'B': ['A', 'B', 'C', 'F'],
                'C': ['B', 'C', 'F'],
                'D': ['D', 'E', 'F'],
                'E': ['A', 'D', 'E', 'F'],
                'F': ['A', 'B', 'C', 'D', 'E', 'F']}

problem2 = Problem(ini_room2, connections2)
for method1, method2 in method_pair:
    problem2.simulate(method1=method1, method2=method2)

method: ('random', 'random')
simulate times: 10000
Total moves: 60484
Average moves: 6.0484
---------
method: ('distribution', 'random')
simulate times: 10000
Total moves: 55191
Average moves: 5.5191
---------
method: ('distribution', 'distribution')
simulate times: 10000
Total moves: 46128
Average moves: 4.6128
---------


In [4]:
ini_room3 = 'A'
connections3 = {'A': ['A', 'B', 'F'],
                'B': ['A', 'B', 'C'],
                'C': ['B', 'C', 'D'],
                'D': ['C', 'D', 'E'],
                'E': ['D', 'E', 'F'],
                'F': ['A', 'E', 'F']}

problem3 = Problem(ini_room2, connections2)
for method1, method2 in method_pair:
    problem3.simulate(method1=method1, method2=method2)

method: ('random', 'random')
simulate times: 10000
Total moves: 60140
Average moves: 6.014
---------
method: ('distribution', 'random')
simulate times: 10000
Total moves: 54275
Average moves: 5.4275
---------
method: ('distribution', 'distribution')
simulate times: 10000
Total moves: 46792
Average moves: 4.6792
---------


In [5]:
problem4 = Problem(ini_room1, connections1)
first_move = ['A', 'B', 'D']

for fm in first_move:
    for method1, method2 in method_pair:
        total_moves = 0
        for _ in range(100000):
            problem4.init_problem(method1, method2)
            agent2_next_room = random.choice(connections1[problem4.agent2.room])
            problem4.execute_a_round(fm, agent2_next_room)
            total_moves += 1
            while problem4.agent1.room != problem4.agent2.room:
                problem4.auto_execute_a_round()
                total_moves += 1
        print(
            f'First move: {fm}\nMethods: {method1, method2}\nSimulate times:{100000}\nTotal moves: {total_moves}\nAverage moves: {total_moves / 100000}\n---------')

First move: A
Methods: ('random', 'random')
Simulate times:100000
Total moves: 448256
Average moves: 4.48256
---------
First move: A
Methods: ('distribution', 'random')
Simulate times:100000
Total moves: 445990
Average moves: 4.4599
---------
First move: A
Methods: ('distribution', 'distribution')
Simulate times:100000
Total moves: 445446
Average moves: 4.45446
---------
First move: B
Methods: ('random', 'random')
Simulate times:100000
Total moves: 447081
Average moves: 4.47081
---------
First move: B
Methods: ('distribution', 'random')
Simulate times:100000
Total moves: 445111
Average moves: 4.45111
---------
First move: B
Methods: ('distribution', 'distribution')
Simulate times:100000
Total moves: 443320
Average moves: 4.4332
---------
First move: D
Methods: ('random', 'random')
Simulate times:100000
Total moves: 449618
Average moves: 4.49618
---------
First move: D
Methods: ('distribution', 'random')
Simulate times:100000
Total moves: 447387
Average moves: 4.47387
---------
First mo